In [1]:
from sqlalchemy import create_engine

# --- MODIFICA ESTO según tu MySQL ---
DB_USER = "root"
DB_PASSWORD = "Nereaivan79."
DB_HOST = "localhost"      # puede ser una IP o nombre de host
DB_PORT = 3306             # por defecto MySQL usa 3306
DB_NAME = "sakila"
# ------------------------------------

# URL de conexión usando pymysql
connection_url = f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

# crear engine
engine = create_engine(connection_url)


DEVUELVE DATAFRAME CON RENTALS DEL MES/AÑO

In [2]:
import pandas as pd

def rentals_month(engine, month, year):
    """
    Recupera las filas de la tabla rental para el mes y año indicados.
    month: int (1-12)
    year: int (ej. 2005)
    Devuelve: DataFrame con columnas rental_id, customer_id, rental_date, inventory_id, return_date
    """
    # formateamos mes y year en la consulta para evitar errores
    query = f"""
    SELECT rental_id, rental_date, inventory_id, return_date, customer_id
    FROM rental
    WHERE MONTH(rental_date) = {int(month)}
      AND YEAR(rental_date) = {int(year)};
    """
    df = pd.read_sql(query, engine)
    return df


CUENTA POR CUSTOMER_ID Y NOMBRA LA COLUMNA DINÁMICAMENTE

In [3]:
def rental_count_month(df, month, year):
    """
    df: DataFrame devuelto por rentals_month
    month: int
    year: int
    Devuelve: DataFrame con columnas [customer_id, rentals_MM_YYYY]
    """
    # nombre de columna con cero relleno para mes
    col_name = f"rentals_{str(month).zfill(2)}_{year}"
    
    # agrupar por customer_id y contar rental_id
    df_grouped = (
        df.groupby("customer_id")["rental_id"]
          .count()
          .reset_index()
          .rename(columns={"rental_id": col_name})
    )
    return df_grouped


COMBINA DOS DATAFRAMES Y CREA UNA COLUNMA "DIFFERENCE"

In [4]:
def compare_rentals(df1, df2, how='inner'):
    """
    df1, df2: DataFrames devueltos por rental_count_month
    how: 'inner' -> solo clientes que aparecen en ambos meses (recomendado para este ejercicio),
         'outer' -> incluir todos los clientes y rellenar NaN con 0
    Devuelve: DataFrame con customer_id, col1, col2, difference (col2 - col1)
    """
    # nombres de la columna de conteo (segunda columna)
    col1 = df1.columns[1]
    col2 = df2.columns[1]
    
    if how == 'outer':
        df_merged = pd.merge(df1, df2, on="customer_id", how="outer")
        df_merged[[col1, col2]] = df_merged[[col1, col2]].fillna(0).astype(int)
    else:
        df_merged = pd.merge(df1, df2, on="customer_id", how="inner")
    
    df_merged["difference"] = df_merged[col2] - df_merged[col1]
    # opcional: ordenar por diferencia descendente o por total en mes2
    df_merged = df_merged.sort_values(by="difference", ascending=False).reset_index(drop=True)
    return df_merged


MAYO

In [7]:
# Obtener los rentals de mayo 2005
df_may = rentals_month(engine, 5, 2005)

# Contar rentals por cliente en mayo 2005
df_may_counts = rental_count_month(df_may, 5, 2005)

df_may_counts.head()


,customer_id,rentals_05_2005
0,1,2
1,2,1
2,3,2
3,5,3
4,6,3


JUNIO

In [8]:
# Obtener los rentals de junio 2005
df_june = rentals_month(engine, 6, 2005)

# Contar rentals por cliente en junio 2005
df_june_counts = rental_count_month(df_june, 6, 2005)

df_june_counts.head()


,customer_id,rentals_06_2005
0,1,7
1,2,1
2,3,4
3,4,6
4,5,5


COMPARAR MAYO VS JUNIO

In [9]:
df_compare = compare_rentals(df_may_counts, df_june_counts)
df_compare.head()


,customer_id,rentals_05_2005,rentals_06_2005,difference
0,454,1,10,9
1,213,1,9,8
2,295,1,9,8
3,457,1,9,8
4,380,1,8,7


VER CLIENTES MÁS ACTIVOS EN JUNIO

In [10]:
df_compare.sort_values("difference", ascending=False).head(10)


,customer_id,rentals_05_2005,rentals_06_2005,difference
0,454,1,10,9
3,457,1,9,8
1,213,1,9,8
2,295,1,9,8
4,380,1,8,7
8,27,1,8,7
6,260,1,8,7
5,234,1,8,7
7,561,2,9,7
22,526,3,9,6
